In [ ]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [ ]:
import azureml.core
# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
vm_size = "STANDARD_D2_V2"
max_nodes = 4
cluster_name = 'clusterfrank'
cluster_config = AmlCompute.provisioning_configuration(
    vm_size = vm_size,
    min_nodes = 0, max_nodes = max_nodes,
    vm_priority = 'dedicated'
)
cluster = ComputeTarget.create(
    ws, cluster_name, cluster_config
)
cluster.wait_for_completion()

In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling, BayesianParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os

# Specify parameter sampler
ps = BayesianParameterSampling({
    '--C': uniform(0.05, 2)
})

# Specify a Policy
policy = BanditPolicy(
    slack_amount = 0.2,
    evaluation_interval = 1,
    delay_evaluation = 5
)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(
    source_directory = './',
    entry_script = 'train.py',
    compute_target = cluster
)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    estimator = est,
    hyperparameter_sampling = ps,
    policy = None,
    primary_metric_name = 'Accuracy',
    primary_metric_goal = PrimaryMetricGoal.MAXIMIZE,
    max_total_runs = 50,
    max_concurrent_runs = 4
)

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
run = exp.submit(config = hyperdrive_config)
RunDetails(run).show()
run.wait_for_completion()

In [ ]:
best_run = run.get_best_run_by_primary_metric()
best_run.get_file_names()

In [ ]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
best_run_arguments = best_run.get_details()['runDefinition']['arguments']

model = best_run.register_model(
    model_name = 'project01model',
    tags = {'training context': 'HyperDrive'},
    properties = {'Accuracy': best_run_metrics['Accuracy'], 
                  'Arguments': best_run_arguments}
)
os.makedirs('outputs', exist_ok=True)
joblib.dump(value=model, filename='outputs/best_model.pkl')

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(### YOUR DATA OBJECT HERE ###)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task=,
    primary_metric=,
    training_data=,
    label_column_name=,
    n_cross_validations=)

In [ ]:
# Submit your automl run

### YOUR CODE HERE ###

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###